In [1]:
!pip install spotipy

In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
client_id= "b2e1e35aac3543e2ab2b6725a6199f53"
client_secret= "3dd14510a87f4a33813336607ea5236c"
redirect_uri='http://google.com'

In [4]:
username='Cv0zrIJcTBK0ydDsIDLDHw'
playlist = '37i9dQZEVXbMDoHDwVN2tF'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
scope = 'user-library-read playlist-read-private'
try:
    token = util.prompt_for_user_token(username, scope,client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
    sp=spotipy.Spotify(auth= token)
except:
    print('Token is not accesible for ' + username)

In [5]:
import argparse
import pprint
import sys
import os
import subprocess
import json
import pandas as pd

#client_credentials_manager = SpotifyClientCredentials()

def get_playlist_content(username, playlist_id, sp):
    offset = 0
    songs = []
    while True:
        content = sp.user_playlist_tracks(username, playlist_id, fields=None,
                                          limit=100, offset=offset, market=None)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break

    with open('{}-{}'.format(username, playlist_id), 'w') as outfile:
        json.dump(songs, outfile)


def get_playlist_audio_features(username, playlist_id, sp):
    offset = 0
    songs = []
    items = []
    ids = []
    while True:
        content = sp.user_playlist_tracks(username, playlist_id, fields=None, limit=100, offset=offset, market=None)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break

    for i in songs:
        ids.append(i['track']['id'])

    index = 0
    audio_features = []
    while index < len(ids):
        audio_features += sp.audio_features(ids[index:index + 50])
        index += 50

    features_list = []
    for features in audio_features:
        features_list.append([features['energy'], features['liveness'],
                              features['tempo'], features['speechiness'],
                              features['acousticness'], features['instrumentalness'],
                              features['time_signature'], features['danceability'],
                              features['key'], features['duration_ms'],
                              features['loudness'], features['valence'],
                              features['mode'], features['type'],
                              features['uri']])

    df = pd.DataFrame(features_list, columns=['energy', 'liveness',
                                              'tempo', 'speechiness',
                                              'acousticness', 'instrumentalness',
                                              'time_signature', 'danceability',
                                              'key', 'duration_ms', 'loudness',
                                              'valence', 'mode', 'type', 'uri'])
    df.to_csv('{}-{}.csv'.format(username, playlist_id), index=False)


def get_user_playlist(username, sp):
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        print("Name: {}, Number of songs: {}, Playlist ID: {} ".
              format(playlist['name'].encode('utf8'),
                     playlist['tracks']['total'],
                     playlist['id']))


In [7]:
get_playlist_audio_features(username,'4gNfyP8v62S9T1o1uWDll3',sp)

In [23]:
get_playlist_content(username,playlist,sp)

In [21]:
get_user_playlist(username,sp)

Name: b'KPOPERA', Number of songs: 34, Playlist ID: 7mgp5snIgJ2W4kvU6YE6EF 
Name: b'80', Number of songs: 22, Playlist ID: 4gNfyP8v62S9T1o1uWDll3 
Name: b'happy days', Number of songs: 48, Playlist ID: 2znMqQDIhpPsWZYbNDmC6q 
Name: b'Esmola espiritual', Number of songs: 387, Playlist ID: 1kBgCUCgPu9VqWw22hwHX2 
Name: b'Tokusatsu & Anime', Number of songs: 50, Playlist ID: 37i9dQZF1DWYYYb1pjoXTj 
Name: b'T\xc3\x98P PLAYLIST', Number of songs: 48, Playlist ID: 1WCRwQ0f4yT8tuIUUxP5m1 
Name: b'jeezz', Number of songs: 9, Playlist ID: 0ofPotHryZgU9mriSHkfM5 
Name: b'pinta gilipollas', Number of songs: 144, Playlist ID: 2ehO6n3kgsKZHYhzuLwaXs 
Name: b'aloka acha que entende de indie e rock', Number of songs: 187, Playlist ID: 6Yc4u6j8fc6PRzH6U7liSf 
Name: b'Bandersnatch Netflix Playlist', Number of songs: 10, Playlist ID: 7uAUInQomxIWcJnuStMp6X 
Name: b'VIE', Number of songs: 25, Playlist ID: 0uLEkmSRFuWqqB9jQwukvn 
Name: b'Tastebreakers', Number of songs: 50, Playlist ID: 37i9dQZF1Ej2tLjARE